In [2]:
from pprint import pprint
from nltk.corpus.reader import ConllCorpusReader
import statistics
import pandas as pd
import spacy

In [3]:
nlp_web = spacy.load("en_core_web_lg")

# Assuming you have the 'conll.testa' file in the 'conll2003' directory
testa = ConllCorpusReader('conll2003/', 'eng.testa', ['words', 'pos', 'ignore', 'chunk'])
testa_sents = list(testa.iob_sents())

# Initialize an empty list to store spans
testa_spans = []

for sent in testa_sents:
    testing = []
    words = [token for token, pos, label in sent]
    spans = [(start) for start, end, label in sent]

    # # # Add spans to the list
    testing.append(spans)
    testa_spans.append(testing)

# pprint(testa_spans)
list_sent = []

for sent in testa_spans: 
    for word in sent:
        sentence = ' '.join(word)
    list_sent.append(sentence)

testa_df = pd.DataFrame(list_sent)
testa_df.columns = ['Document']

ner_results = []

for document in testa_df['Document']:
    text = str(document)
    # print(text)
    doc_web = nlp_web(text)
    ner_results.append(doc_web.ents)

testa_df['ner_results'] = ner_results

display(testa_df['ner_results'])

0                                                      ()
1                                                      ()
2                        ((LONDON), (1996, -, 08, -, 30))
3       ((West, Indian), (Phil, Simmons), (four), (38)...
4       ((Essex), (Derbyshire), (Surrey), (Kent), (Not...
                              ...                        
3461                                                   ()
3462    ((Bangladesh, Lamps), (Chittagong, Cement), (A...
3463                                                   ()
3464    ((DSE), (2.73), (0.22, percent), (1,196.35), (...
3465          ((Dhaka, Newsroom), (880, -, 2, -, 506363))
Name: ner_results, Length: 3466, dtype: object

In [80]:

ent_type_spans = {}

please = []

for ner_result in testa_df['ner_results']:
    testing = []
    for ent in ner_result:
        ent_type = ent.label_
        span_text = ent.text
        if ent_type == "PERSON":
            ent_type = "PER"
            testing.append(ent_type)
        elif ent_type == "LOC" or ent_type == "ORG":
            testing.append(ent_type)
  
        
    please.append(testing)
    
    
pprint(please)
    

[[],
 [],
 [],
 ['PER'],
 ['PER'],
 ['PER'],
 ['PER'],
 ['PER', 'PER'],
 ['PER'],
 [],
 ['LOC', 'PER'],
 ['PER', 'ORG'],
 [],
 ['PER', 'PER', 'PER'],
 ['PER'],
 ['PER', 'PER'],
 ['PER'],
 [],
 [],
 [],
 [],
 [],
 ['PER'],
 [],
 ['PER', 'ORG', 'ORG', 'PER', 'PER'],
 ['ORG', 'PER', 'PER', 'ORG'],
 ['ORG', 'PER', 'ORG', 'PER', 'PER'],
 ['PER', 'PER', 'PER'],
 ['PER', 'PER', 'PER'],
 ['PER', 'PER', 'ORG'],
 ['ORG', 'PER', 'PER', 'PER'],
 [],
 ['ORG'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['ORG'],
 ['LOC'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['ORG'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['PER'],
 ['PER', 'PER'],
 ['PER', 'PER'],
 ['PER', 'PER', 'PER'],
 ['PER', 'ORG'],
 ['PER'],
 [],
 [],
 ['ORG', 'PER'],
 ['ORG', 'PER'],
 [],
 ['ORG'],
 [],
 [],
 [],
 ['ORG'],
 [],
 [],
 ['ORG'],
 ['ORG'],
 [],
 [],
 ['PER'],
 [],
 [],
 [],
 ['PER'],
 [],
 [],
 ['PER', 'PER'],
 ['PER'

In [92]:

testa = ConllCorpusReader('conll2003/', 'eng.testa', ['words', 'pos', 'ignore', 'chunk'])
testa_sents = list(testa.iob_sents())

def sent_to_labels(sent: list) -> list:
    return [label for word, pos, label in sent if label != 'O']

testa_labels = [sent_to_labels(sent) for sent in testa_sents]

for doc in testa_labels:
    for i, ent in enumerate(doc):
        if "-" in ent:
            doc[i] = ent[2:]

# Replace 'MISC' with an empty list
testa_labels = [item if 'MISC' not in item else [] for item in testa_labels]





In [96]:
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(testa_labels)
binary_labels2 = mlb.fit_transform(please)

# Compute classification report
report = metrics.classification_report(binary_labels, binary_labels2)
# report = metrics.classification_report(testa_labels, please)

pprint(report)
# LOC
# ORG
# PER

('              precision    recall  f1-score   support\n'
 '\n'
 '           0       0.45      0.02      0.04      1015\n'
 '           1       0.48      0.54      0.51       782\n'
 '           2       0.70      0.88      0.78       867\n'
 '\n'
 '   micro avg       0.60      0.45      0.52      2664\n'
 '   macro avg       0.55      0.48      0.44      2664\n'
 'weighted avg       0.54      0.45      0.42      2664\n'
 ' samples avg       0.30      0.24      0.26      2664\n')


/Users/sungheepark/miniconda3/envs/info498/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sungheepark/miniconda3/envs/info498/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
